In [1]:
!wget https://github.com/anthdm/ml-email-clustering/blob/master/split_emails.csv

--2019-03-01 15:30:32--  https://github.com/anthdm/ml-email-clustering/blob/master/split_emails.csv
Resolving github.com (github.com)... 192.30.255.113, 192.30.255.112
Connecting to github.com (github.com)|192.30.255.113|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/html]
Saving to: ‘split_emails.csv’

split_emails.csv        [ <=>                ]  51.67K  --.-KB/s    in 0.07s   

2019-03-01 15:30:33 (774 KB/s) - ‘split_emails.csv’ saved [52912]



In [7]:
!wget https://www.dropbox.com/s/p82abqm8dg0ihqy/emails.csv.zip

--2019-03-01 16:00:50--  https://www.dropbox.com/s/p82abqm8dg0ihqy/emails.csv.zip
Resolving www.dropbox.com (www.dropbox.com)... 162.125.8.1, 2620:100:601b:1::a27d:801
Connecting to www.dropbox.com (www.dropbox.com)|162.125.8.1|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/p82abqm8dg0ihqy/emails.csv.zip [following]
--2019-03-01 16:00:50--  https://www.dropbox.com/s/raw/p82abqm8dg0ihqy/emails.csv.zip
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uca4a18f264bf8f11ca889bac639.dl.dropboxusercontent.com/cd/0/inline/AcQnOEs3w_1433vw3BG5devj2duJe_fAI0u5RCv0p1HGvbrp_6oJURWDNRO5jN8vPX2kFMqbqe1FdZSzES1RyM65YrLYWWEJgjwZYbPEmCE9QA/file# [following]
--2019-03-01 16:00:51--  https://uca4a18f264bf8f11ca889bac639.dl.dropboxusercontent.com/cd/0/inline/AcQnOEs3w_1433vw3BG5devj2duJe_fAI0u5RCv0p1HGvbrp_6oJURWDNRO5jN8vPX2kFMqbqe1FdZSzES1RyM65YrLYWWEJgjwZYbPEmCE9QA/file
Resolving uca

In [8]:
!unzip emails.csv.zip

Archive:  emails.csv.zip
  inflating: emails.csv              


In [1]:
import pandas as pd

In [2]:
emails = pd.read_csv('emails.csv')
email_subset = emails[:50000]

print(email_subset.shape)
print(email_subset.head())

(50000, 2)
                       file                                            message
0     allen-p/_sent_mail/1.  Message-ID: <18782981.1075855378110.JavaMail.e...
1    allen-p/_sent_mail/10.  Message-ID: <15464986.1075855378456.JavaMail.e...
2   allen-p/_sent_mail/100.  Message-ID: <24216240.1075855687451.JavaMail.e...
3  allen-p/_sent_mail/1000.  Message-ID: <13505866.1075863688222.JavaMail.e...
4  allen-p/_sent_mail/1001.  Message-ID: <30922949.1075863688243.JavaMail.e...


In [3]:
def parse_raw_message(raw_message):
    lines = raw_message.split('\n')
    email = {}
    message = ''
    keys_to_extract = ['from', 'to']
    for line in lines:
        if ':' not in line:
            message += line.strip()
            email['body'] = message
        else:
            pairs = line.split(':')
            key = pairs[0].lower()
            val = pairs[1].strip()
            if key in keys_to_extract:
                email[key] = val
    return email

In [4]:
def parse_into_emails(messages):
    emails = [parse_raw_message(message) for message in messages]
    return {
        'body': map_to_list(emails, 'body'),
        'to': map_to_list(emails, 'to'),
        'from_': map_to_list(emails, 'from')
    }


In [5]:
def map_to_list(emails, key):
    results = []
    for email in emails:
        if key not in email:
            results.append('')
        else:
            results.append(email[key])
    return results

In [6]:
email_df = pd.DataFrame(parse_into_emails(email_subset.message))
print(email_df.head())

                                                body                       to  \
0                               Here is our forecast     tim.belden@enron.com   
1  Traveling to have a business meeting takes the...  john.lavorato@enron.com   
2                     test successful.  way to go!!!   leah.arsdall@enron.com   
3  Randy,Can you send me a schedule of the salary...    randall.gay@enron.com   
4                                                        greg.piper@enron.com   

                     from_  
0  phillip.allen@enron.com  
1  phillip.allen@enron.com  
2  phillip.allen@enron.com  
3  phillip.allen@enron.com  
4  phillip.allen@enron.com  


In [7]:
import re
import numpy as np

In [8]:
import gensim

In [9]:
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

In [10]:
# spacy for lemmatization
import spacy

In [11]:
# for plotting
import pyLDAvis
import pyLDAvis.gensim
import matplotlib.pyplot as plt

In [12]:
#  >>> import nltk
#  >>> nltk.download('stopwords')

In [13]:
# prep NLTK Stop words
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use'])

/opt/anaconda3/lib/python3.7/site-packages/nltk/decorators.py:68: DeprecationWarning: `formatargspec` is deprecated since Python 3.5. Use `signature` and the `Signature` object directly
  regargs, varargs, varkwargs, defaults, formatvalue=lambda value: ""


In [14]:
print(email_df.iloc[2]['body']) # displays info below

test successful.  way to go!!!


In [15]:
# Convert email body to list
data = email_df.body.values.tolist()

In [16]:
# tokenize - break down each sentence into a list of words
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations


In [17]:
data_words = list(sent_to_words(data))

In [18]:
print(data_words[3])

['randy', 'can', 'you', 'send', 'me', 'schedule', 'of', 'the', 'salary', 'and', 'level', 'of', 'everyone', 'in', 'thescheduling', 'group', 'plus', 'your', 'thoughts', 'on', 'any', 'changes', 'that', 'need', 'to', 'be', 'made', 'patti', 'for', 'example', 'phillip']


In [19]:
from gensim.models.phrases import Phrases, Phraser

In [20]:
# Build the bigram and trigram models
bigram = Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = Phrases(bigram[data_words], threshold=100)

/opt/anaconda3/lib/python3.7/site-packages/gensim/models/phrases.py:494: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")


In [21]:
# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = Phraser(bigram)
trigram_mod = Phraser(trigram)

In [22]:
# See trigram example
print(trigram_mod[bigram_mod[data_words[200]]])

['forwarded', 'by', 'phillip_allen', 'hou', 'ect', 'on', 'hello', 'men', 'have', 'attached', 'my', 'worksheet', 'in', 'case', 'you', 'want', 'to', 'review', 'the', 'data', 'while_amon_holiday', 'thanks']


In [23]:
# remove stop_words, make bigrams and lemmatize
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent))
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [24]:
# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)

In [25]:
# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)

In [26]:
#python -m spacy download en

In [27]:
# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
nlp = spacy.load('en', disable=['parser', 'ner'])

In [28]:
# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

In [29]:
print(data_lemmatized[200])

['forward', 'phillip_allen', 'ect', 'man', 'attach', 'worksheet', 'case', 'want', 'review', 'datum', 'amon_holiday', 'thank']


In [30]:
# create dictionary and corpus both are needed for (LDA) topic modeling

# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)

# Create Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]


In [31]:
!wget http://mallet.cs.umass.edu/dist/mallet-2.0.8.zip

--2019-04-19 18:25:38--  http://mallet.cs.umass.edu/dist/mallet-2.0.8.zip
Resolving mallet.cs.umass.edu (mallet.cs.umass.edu)... 128.119.246.70
Connecting to mallet.cs.umass.edu (mallet.cs.umass.edu)|128.119.246.70|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 16184794 (15M) [application/zip]
Saving to: ‘mallet-2.0.8.zip.4’

mallet-2.0.8.zip.4  100%[===================>]  15.43M  10.9MB/s    in 1.4s    

2019-04-19 18:25:40 (10.9 MB/s) - ‘mallet-2.0.8.zip.4’ saved [16184794/16184794]



In [32]:
#!unzip mallet-2.0.8.zip

In [33]:
mallet_path = './mallet-2.0.8/bin/mallet'

In [34]:
import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

In [35]:
# Build LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=20,
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

In [36]:
# topic modeling
# corpus, dictionary and number of topics required for LDA
# alpha and eta are hyperparameters that affect sparsity of the topics
# chunksize is the number of documents to be used in each training chunk
# update_every determines how often the model parameters should be updated
# passes is the total number of training passes
# Print the Keyword in the 10 topics

In [37]:
print(lda_model.print_topics())# The weights reflect how important a keyword is to that topic.

[(0, '0.033*"table" + 0.030*"york" + 0.028*"document" + 0.025*"hope" + 0.019*"right" + 0.018*"size" + 0.014*"title" + 0.012*"age" + 0.011*"model" + 0.011*"role"'), (1, '0.241*"ect" + 0.138*"corp" + 0.068*"hou" + 0.010*"forward" + 0.008*"kayne_coulter" + 0.007*"scott" + 0.005*"elizabeth_sager" + 0.005*"sturm" + 0.005*"highlight" + 0.005*"louis_soldano"'), (2, '0.121*"page" + 0.035*"law" + 0.019*"free" + 0.014*"click" + 0.013*"new" + 0.011*"offer" + 0.011*"president" + 0.009*"find" + 0.008*"appeal" + 0.008*"appropriate"'), (3, '0.565*"enron" + 0.039*"michelle" + 0.009*"communication" + 0.008*"smith" + 0.007*"ee" + 0.007*"rick" + 0.006*"david" + 0.005*"johnson" + 0.004*"brown" + 0.004*"ryan"'), (4, '0.040*"agreement" + 0.029*"pay" + 0.024*"cash" + 0.020*"claim" + 0.016*"legal" + 0.013*"contract" + 0.013*"payment" + 0.012*"permit" + 0.011*"act" + 0.010*"rate"'), (5, '0.077*"union" + 0.058*"image" + 0.013*"judge" + 0.011*"protection" + 0.010*"trial" + 0.010*"bush" + 0.009*"register" + 0.008

In [38]:
doc_lda = lda_model[corpus]

In [39]:
# Model perplexity and topic coherence provide a convenient
# measure to judge how good a given topic model is.
# Compute Perplexity
print('\nPerplexity: ', lda_model.log_perplexity(corpus))  # a measure of how good the model is. lower the better.



Perplexity:  -9.698512656430935


In [ ]:
# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)



Coherence Score:  0.44502925615763134


In [ ]:
# Visualize the topics
pyLDAvis.enable_notebook(sort=True)
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)

/opt/anaconda3/lib/python3.7/site-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


In [ ]:
pyLDAvis.display(vis)

In [ ]:
# now using mallet

In [ ]:
mallet_path = 'mallet-2.0.8/bin/mallet'

In [ ]:
ldamallet = gensim.models.wrappers.LdaMallet(mallet_path, corpus=corpus, num_topics=20, id2word=id2word)

In [ ]:
# Show Topics
print(ldamallet.show_topics(formatted=False))

[(1, [('game', 0.029903769481178316), ('week', 0.026814218223493738), ('play', 0.016828194084409626), ('update', 0.015547788012769764), ('team', 0.012518255789693303), ('start', 0.01217528987764691), ('sunday', 0.009723083606515208), ('season', 0.009500155763685054), ('good', 0.009157189851638662), ('pass', 0.008539851209955157)]), (9, [('review', 0.017545058220216313), ('issue', 0.01465432947475843), ('message', 0.013001874490279551), ('discuss', 0.01278410910472807), ('work', 0.012767029466645602), ('meeting', 0.012378467700269432), ('attach', 0.012190591681362273), ('original', 0.011725171543614991), ('plan', 0.01094804801086265), ('agreement', 0.01037161022557932)]), (8, [('enron', 0.5446866155283804), ('baughman', 0.006719463249678985), ('smith', 0.006665680652382905), ('david', 0.005287501596670857), ('joe_stepenovitch', 0.004016887735550969), ('ryan', 0.003932852427275845), ('tom', 0.003882431242310769), ('dean_laurent', 0.0037815888723806194), ('larry_campbell', 0.0037109992134

In [ ]:
# Compute Coherence Score
coherence_model_ldamallet = CoherenceModel(model=ldamallet, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_ldamallet = coherence_model_ldamallet.get_coherence()
print('\nCoherence Score: ', coherence_ldamallet)


Coherence Score:  0.6235682212776862


In [ ]:
# # Visualize the topics with mallet model
# pyLDAvis.enable_notebook(sort=True)
# vis = pyLDAvis.gensim.prepare(ldamallet, corpus, id2word)
# pyLDAvis.display(vis)

In [ ]:
def compute_coherence_values(dictionary, corpus, texts, limit, start=2, step=3):
    """
    Compute c_v coherence for various number of topics

    Parameters:
    ----------
    dictionary : Gensim dictionary
    corpus : Gensim corpus
    texts : List of input texts
    limit : Max num of topics

    Returns:
    -------
    model_list : List of LDA topic models
    coherence_values : Coherence values corresponding to the LDA model with respective number of topics
    """
    coherence_values = []
    model_list = []
    for num_topics in range(start, limit, step):
        model = gensim.models.wrappers.LdaMallet(mallet_path, corpus=corpus, num_topics=num_topics, id2word=id2word)
        model_list.append(model)
        coherencemodel = CoherenceModel(model=model, texts=texts, dictionary=dictionary, coherence='c_v')
        coherence_values.append(coherencemodel.get_coherence())

    return model_list, coherence_values

In [ ]:
# run
model_list, coherence_values = compute_coherence_values(dictionary=id2word, corpus=corpus, texts=data_lemmatized, start=2, limit=40, step=6)


In [ ]:
# Show graph
limit=40; start=2; step=6;
x = range(start, limit, step)
plt.plot(x, coherence_values)
plt.xlabel("Num Topics")
plt.ylabel("Coherence score")
plt.legend(("coherence_values"), loc='best')
plt.show()

In [ ]:
# Print the coherence scores
for m, cv in zip(x, coherence_values):
    print("Num Topics =", m, " has Coherence Value of", round(cv, 4))

In [ ]:
# Select the model and print the topics
optimal_model = model_list[2]
model_topics = optimal_model.show_topics(formatted=False)
print(optimal_model.print_topics(num_words=10))

In [ ]:
def format_topics_sentences(ldamodel=lda_model, corpus=corpus, texts=data):
    # Init output
    sent_topics_df = pd.DataFrame()

    # Get main topic in each document
    for i, row in enumerate(ldamodel[corpus]):
        row = sorted(row, key=lambda x: (x[1]), reverse=True)
        # Get the Dominant topic, Perc Contribution and Keywords for each document
        for j, (topic_num, prop_topic) in enumerate(row):
            if j == 0:  # => dominant topic
                wp = ldamodel.show_topic(topic_num)
                topic_keywords = ", ".join([word for word, prop in wp])
                sent_topics_df = sent_topics_df.append(pd.Series([int(topic_num), round(prop_topic,4), topic_keywords]), ignore_index=True)
            else:
                break
    sent_topics_df.columns = ['Dominant_Topic', 'Perc_Contribution', 'Topic_Keywords']

    # Add original text to the end of the output
    contents = pd.Series(texts)
    sent_topics_df = pd.concat([sent_topics_df, contents], axis=1)
    return(sent_topics_df)



In [ ]:
df_topic_sents_keywords = format_topics_sentences(ldamodel=optimal_model, corpus=corpus, texts=data)

In [ ]:
# Format
df_dominant_topic = df_topic_sents_keywords.reset_index()
df_dominant_topic.columns = ['Document_No', 'Dominant_Topic', 'Topic_Perc_Contrib', 'Keywords', 'Text']

In [ ]:
# Show
df_dominant_topic.head(10)

In [ ]:
df_dominant_topic.Keywords.iloc[1]

In [ ]:
df_dominant_topic.Text.iloc[1]

In [ ]:
# Group top 5 sentences under each topic
sent_topics_sorteddf_mallet = pd.DataFrame()

sent_topics_outdf_grpd = df_topic_sents_keywords.groupby('Dominant_Topic')

for i, grp in sent_topics_outdf_grpd:
    sent_topics_sorteddf_mallet = pd.concat([sent_topics_sorteddf_mallet, 
                                             grp.sort_values(['Perc_Contribution'], ascending=[0]).head(1)], 
                                            axis=0)

In [ ]:
# Reset Index    
sent_topics_sorteddf_mallet.reset_index(drop=True, inplace=True)

In [ ]:
# Format
sent_topics_sorteddf_mallet.columns = ['Topic_Num', "Topic_Perc_Contrib", "Keywords", "Text"]

In [ ]:
# Show
sent_topics_sorteddf_mallet

In [ ]:
sent_topics_sorteddf_mallet.Text.iloc[6]

In [ ]:
# Number of Documents for Each Topic
topic_counts = df_topic_sents_keywords['Dominant_Topic'].value_counts()

In [ ]:
# Percentage of Documents for Each Topic
topic_contribution = round(topic_counts/topic_counts.sum(), 4)

In [ ]:
# Topic Number and Keywords
topic_num_keywords = df_topic_sents_keywords[['Dominant_Topic', 'Topic_Keywords']]

In [ ]:
# Concatenate Column wise
df_dominant_topics = pd.concat([topic_num_keywords, topic_counts, topic_contribution], axis=1)

In [ ]:
# Change Column names
df_dominant_topics.columns = ['Dominant_Topic', 'Topic_Keywords', 'Num_Documents', 'Perc_Documents']

In [ ]:
# Show
df_dominant_topics